# Setup

In [ ]:
import ee
import palettable
import pandas as pd

## Authenticate to Earth Engine Servers

In [ ]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=n6rv6auJLS8w5Sg6bidrOlQpkaBplWwfXDgAp_Lz_kM&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g63ztj8E_cBXeb5brNX3XNqhJcECtfLXoMAyCiQKM2PrjLKMuNiFfQ

Successfully saved authorization token.


# Analysis

## Examine the ERA5-Land image collection

Reference the image collection for the ERA5-Land hourly dataset

In [ ]:
era5land = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")

Count the number of available records.

In [ ]:
print(
  f'The ERA5 Land Hourly dataset has {era5land.size().getInfo()} records.'
)

The ERA5 Land Hourly dataset has 349894 records.


Find the earliest and latest data in the collection.

In [ ]:
min_date = ee.Date(
    era5land.aggregate_min('system:time_start')
  ).format().getInfo()
max_date = ee.Date(
    era5land.aggregate_max('system:time_end')
  ).format().getInfo()
print(
    f'Timestamps in the ERA5 Land dataset range from {min_date} to {max_date}.'
)

Timestamps in the ERA5 Land dataset range from 1981-01-01T02:00:00 to 2020-12-01T00:00:00.


## Examine a sample record

In [ ]:
# Get a sample record.
sample = era5land.filterDate('2020', '2020-01-02').first()
sample.getInfo()

{'bands': [{'crs': 'EPSG:4326',
   'crs_transform': [0.1, 0, -180.05, 0, -0.1, 90.05],
   'data_type': {'precision': 'double', 'type': 'PixelType'},
   'dimensions': [3601, 1801],
   'id': 'dewpoint_temperature_2m'},
  {'crs': 'EPSG:4326',
   'crs_transform': [0.1, 0, -180.05, 0, -0.1, 90.05],
   'data_type': {'precision': 'double', 'type': 'PixelType'},
   'dimensions': [3601, 1801],
   'id': 'temperature_2m'},
  {'crs': 'EPSG:4326',
   'crs_transform': [0.1, 0, -180.05, 0, -0.1, 90.05],
   'data_type': {'precision': 'double', 'type': 'PixelType'},
   'dimensions': [3601, 1801],
   'id': 'skin_temperature'},
  {'crs': 'EPSG:4326',
   'crs_transform': [0.1, 0, -180.05, 0, -0.1, 90.05],
   'data_type': {'precision': 'double', 'type': 'PixelType'},
   'dimensions': [3601, 1801],
   'id': 'soil_temperature_level_1'},
  {'crs': 'EPSG:4326',
   'crs_transform': [0.1, 0, -180.05, 0, -0.1, 90.05],
   'data_type': {'precision': 'double', 'type': 'PixelType'},
   'dimensions': [3601, 1801],
   

In [ ]:
# Print out the band names.
sample.bandNames().getInfo()

['dewpoint_temperature_2m',
 'temperature_2m',
 'skin_temperature',
 'soil_temperature_level_1',
 'soil_temperature_level_2',
 'soil_temperature_level_3',
 'soil_temperature_level_4',
 'lake_bottom_temperature',
 'lake_ice_depth',
 'lake_ice_temperature',
 'lake_mix_layer_depth',
 'lake_mix_layer_temperature',
 'lake_shape_factor',
 'lake_total_layer_temperature',
 'snow_albedo',
 'snow_cover',
 'snow_density',
 'snow_depth',
 'snow_depth_water_equivalent',
 'snowfall',
 'snowmelt',
 'temperature_of_snow_layer',
 'skin_reservoir_content',
 'volumetric_soil_water_layer_1',
 'volumetric_soil_water_layer_2',
 'volumetric_soil_water_layer_3',
 'volumetric_soil_water_layer_4',
 'forecast_albedo',
 'surface_latent_heat_flux',
 'surface_net_solar_radiation',
 'surface_net_thermal_radiation',
 'surface_sensible_heat_flux',
 'surface_solar_radiation_downwards',
 'surface_thermal_radiation_downwards',
 'evaporation_from_bare_soil',
 'evaporation_from_open_water_surfaces_excluding_oceans',
 'evap

## Image Output

In [ ]:
from IPython.display import Image

aoi = ee.Geometry.Rectangle(-84, -56, -31, 14)

url = (
    era5land.filterDate('2020-03-01', '2020-03-02')
        .first()
        .getThumbURL({
          'bands': ['total_precipitation_hourly'],
          'min': 0,
          'max': 5e-3,
          'palette': palettable.scientific.sequential.Bilbao_10.hex_colors,
          'region':aoi,
          'dimensions':'500',
          'format':'jpg'
        })
)
display(Image(url=url))

## Video Output

In [ ]:
url = (
    era5land
    .filterDate('2020', '2020-01-03')
    .getVideoThumbURL({
      'bands': ['total_precipitation_hourly'],
      'min': 0,
      'max': 5e-3,
      'palette': palettable.scientific.sequential.Bilbao_10.hex_colors,
      'region':aoi,
      'dimensions':'500'
    })
)
display(Image(url=url))

### Specifying the projection

In [ ]:
aoi = ee.Geometry.Rectangle(-84, -56, -31, 14)

url = (
    era5land.filterDate('2020', '2020-01-03')
    .getVideoThumbURL({
      'bands': ['total_precipitation_hourly'],
      'min': 0,
      'max': 10e-3,
      #'crs': 'EPSG:32621',  # WGS 84 / UTM zone 21N
      #'crs': 'EPSG:5641',  # Brazil Mercator
      'crs': 'EPSG:3031',  # WGS 84 / Antarctic Polar Stereographic
      'palette': palettable.scientific.sequential.Bilbao_10.hex_colors,
      'region':aoi,
      'dimensions':'500'
    })
)
display(Image(url=url))

## Interactive Map Output

In [ ]:
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
# Set visualization parameters.
viz_params = {
  'bands': ['total_precipitation'],
  'min': 0,
  'max': 5e-3,
  'palette': palettable.matplotlib.Inferno_10.hex_colors
}

# Create a folium map object.
my_map = folium.Map(location=[20, 0], zoom_start=3, height=500)

# Add the elevation model to the map object.
my_map.add_ee_layer(sample, viz_params, 'Total Precipitation')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

# Where to go next...

For additional examples of time series analysis using [pandas](https://pandas.pydata.org/) and [Altair](https://altair-viz.github.io/) visualization libraries, see this notebook tutorial:

https://colab.sandbox.google.com/github/google/earthengine-community/blob/master/tutorials/time-series-visualization-with-altair/index.ipynb